<a href="https://colab.research.google.com/github/anshupandey/AI_Agents/blob/main/AAP_C6_LangChain_Chain_reflection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Reflection


In the context of LLM agent building, reflection refers to the process of prompting an LLM to observe its past steps (along with potential observations from tools/the environment) to assess the quality of the chosen actions.
This is then used downstream for things like re-planning, search, or evaluation.

![Reflection](../img/reflection.png)

This notebook demonstrates a very simple form of reflection in LangGraph.

#### Prerequisites

We will be using a basic agent with a search tool here.

In [1]:
!pip install -q -U langchain-core langchain-community langgraph
!pip install --upgrade --quiet google-cloud-aiplatform requests
!pip install -q -U langchain-google-vertexai wikipedia tavily-python
!pip install -q -U httpx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.4/91.4 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 366.3/366.3 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.8/81.8 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.5/328.5 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.6/983.6 kB 32.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

In [1]:
import sys

if "google.colab" in sys.modules:
    from google.colab import auth

    auth.authenticate_user()

In [2]:
PROJECT_ID = "jrproject-402905"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}

import vertexai

vertexai.init(project=PROJECT_ID, location=LOCATION)

## Generate

For our example, we will create a "5 paragraph essay" generator. First, create the generator:


In [3]:
from langchain_core.messages import AIMessage, BaseMessage, HumanMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_google_vertexai import ChatVertexAI

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are an essay assistant tasked with writing excellent 5-paragraph essays."
            " Generate the best essay possible for the user's request."
            " If the user provides critique, respond with a revised version of your previous attempts.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)


llm = ChatVertexAI(model="gemini-1.5-flash-001")

generate = prompt | llm

In [4]:
essay = ""
request = HumanMessage(
    content="Write an essay on why the little prince is relevant in modern childhood"
)
for chunk in generate.stream({"messages": [request]}):
    print(chunk.content, end="")
    essay += chunk.content

## The Enduring Relevance of the Little Prince in Modern Childhood

Antoine de Saint-Exupéry's "The Little Prince" is a timeless tale that continues to resonate with readers of all ages, particularly children. While the world has changed drastically since its publication in 1943, the story's themes of innocence, wonder, and the search for meaning remain profoundly relevant to the modern child navigating a complex and often overwhelming world. 

The Little Prince's journey across the cosmos, encountering diverse characters and unique planets, mirrors the child's own exploration of the world. Each encounter, from the conceited man

### Reflect

In [5]:
reflection_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a teacher grading an essay submission. Generate critique and recommendations for the user's submission."
            " Provide detailed recommendations, including requests for length, depth, style, etc.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)
reflect = reflection_prompt | llm

In [ ]:
reflection = ""
for chunk in reflect.stream({"messages": [request, HumanMessage(content=essay)]}):
    print(chunk.content, end="")
    reflection += chunk.content

### Repeat

And... that's all there is too it! You can repeat in a loop for a fixed number of steps, or use an LLM (or other check) to decide when the finished product is good enough.

In [ ]:
for chunk in generate.stream(
    {"messages": [request, AIMessage(content=essay), HumanMessage(content=reflection)]}
):
    print(chunk.content, end="")

Title: The Relevance of The Little Prince in Modern Childhood: A Contemporary Analysis

In the digital age, where human connections are often overshadowed by virtual communication, Antoine de Saint-Exupéry's The Little Prince remains a timeless classic that offers invaluable insights for modern children. This essay aims to delve deeper into the relevance of this novella in contemporary childhood, focusing on the complex nature of human relationships, self-discovery, and the inevitability of loss.

Firstly, The Little Prince offers a powerful critique of the superficiality that permeates the digital world. Through the little prince's encounters with various grown-ups, the book emphasizes the importance of genuine connections and understanding. Despite being published in 1943, Saint-Exupéry's work uncannily predicts the isolating effects of technology on human interaction. It encourages children to seek deeper connections, fostering empathy and emotional intelligence. For instance, the l

## Define graph

Now that we've shown each step in isolation, we can wire it up in a graph.

In [ ]:
from typing import List, Sequence

from langgraph.graph import END, MessageGraph, START


async def generation_node(state: Sequence[BaseMessage]):
    return await generate.ainvoke({"messages": state})


async def reflection_node(messages: Sequence[BaseMessage]) -> List[BaseMessage]:
    # Other messages we need to adjust
    cls_map = {"ai": HumanMessage, "human": AIMessage}
    # First message is the original user request. We hold it the same for all nodes
    translated = [messages[0]] + [
        cls_map[msg.type](content=msg.content) for msg in messages[1:]
    ]
    res = await reflect.ainvoke({"messages": translated})
    # We treat the output of this as human feedback for the generator
    return HumanMessage(content=res.content)


builder = MessageGraph()
builder.add_node("generate", generation_node)
builder.add_node("reflect", reflection_node)
builder.add_edge(START, "generate")


def should_continue(state: List[BaseMessage]):
    if len(state) > 6:
        # End after 3 iterations
        return END
    return "reflect"


builder.add_conditional_edges("generate", should_continue)
builder.add_edge("reflect", "generate")
graph = builder.compile()

In [ ]:
async for event in graph.astream(
    [
        HumanMessage(
            content="Generate an essay on the topicality of The Little Prince and its message in modern life"
        )
    ],
):
    print(event)
    print("---")

{'generate': AIMessage(content="Title: The Enduring Relevance of The Little Prince: A Timeless Message for Modern Life\n\nIntroduction:\nAntoine de Saint-Exupéry's The Little Prince is a canonical work of literature, beloved by generations since its publication in 1943. The novella has been translated into more than 250 languages and sold over 140 million copies, making it one of the best-selling books of all time. Its enchanting story transcends cultural boundaries and continues to captivate audiences of all ages. The Little Prince's timeless message remains relevant in modern life, offering insightful commentary on themes such as love, loneliness, responsibility, and the superficiality of the adult world. In this essay, we will discuss the topicality of The Little Prince and its enduring message in today's fast-paced, digitally-connected society.\n\nBody Paragraph 1 - Love and Loneliness:\nOne of the most enduring aspects of The Little Prince is its exploration of love and relationsh

In [ ]:
ChatPromptTemplate.from_messages(event[END]).pretty_print()

================================ Human Message =================================

Generate an essay on the topicality of The Little Prince and its message in modern life

================================== Ai Message ==================================

Title: The Enduring Relevance of The Little Prince: A Timeless Message for Modern Life

Introduction:
Antoine de Saint-Exupéry's The Little Prince is a canonical work of literature, beloved by generations since its publication in 1943. The novella has been translated into more than 250 languages and sold over 140 million copies, making it one of the best-selling books of all time. Its enchanting story transcends cultural boundaries and continues to captivate audiences of all ages. The Little Prince's timeless message remains relevant in modern life, offering insightful commentary on themes such as love, loneliness, responsibility, and the superficiality of the adult world. In this essay, we will discuss the topicality of The Little Princ

## Conclusion

Now that you've applied reflection to an LLM agent, I'll note one thing: self-reflection is inherently cyclic: it is much more effective if the reflection step has additional context or feedback (from tool observations, checks, etc.). If, like in the scenario above, the reflection step simply prompts the LLM to reflect on its output, it can still benefit the output quality (since the LLM then has multiple "shots" at getting a good output), but it's less guaranteed.
